Arrow manages data in arrays (pyarrow.Array), which can be grouped in tables (pyarrow.Table) to represent columns of data in tabular data.



In [1]:
import pyarrow as pa

In [2]:
days = pa.array([1, 12, 17, 23, 28], type=pa.int8())
days

[
  1,
  12,
  17,
  23,
  28
]

Multiple arrays can be combined in tables to form the columns in tabular data when attached to a column name



In [3]:
months = pa.array([1, 3, 5, 7, 1], type=pa.int8())
years = pa.array([1990, 2000, 1995, 2000, 1995], type=pa.int16())

In [4]:
birthdays_table = pa.table([days, months, years], names=["days", "months", "years"])
birthdays_table

pyarrow.Table
days: int8
months: int8
years: int16
----
days: [[1,12,17,23,28]]
months: [[1,3,5,7,1]]
years: [[1990,2000,1995,2000,1995]]

## Saving and Loading Tables

In [5]:
import pyarrow.parquet as pq

pq.write_table(birthdays_table, "birthdays.parquet")

In [6]:
reloaded_birthdays = pq.read_table("birthdays.parquet")
reloaded_birthdays

pyarrow.Table
days: int8
months: int8
years: int16
----
days: [[1,12,17,23,28]]
months: [[1,3,5,7,1]]
years: [[1990,2000,1995,2000,1995]]

##  Performing Computations

In [7]:
import pyarrow.compute as pc

pc.value_counts(birthdays_table["years"])

-- is_valid: all not null
-- child 0 type: int16
  [
    1990,
    2000,
    1995
  ]
-- child 1 type: int64
  [
    1,
    2,
    2
  ]

In [8]:
a = pa.array([1, 1, 2, 3])
pc.sum(a)

<pyarrow.Int64Scalar: 7>

In [9]:
a = pa.array([1, 1, 2, 3])
b = pa.array([4, 1, 2, 8])

pc.equal(a, b)

[
  false,
  true,
  true,
  false
]

In [10]:
x, y = pa.scalar(7.8), pa.scalar(9.3)

pc.multiply(x, y)

<pyarrow.DoubleScalar: 72.54>

In [11]:
t = pa.table({"x": [1, 2, 3], "y": [3, 2, 1]})
i = pc.sort_indices(t, sort_keys=[("y", "ascending")])

i

[
  2,
  1,
  0
]

In [12]:
t = pa.table(
    [
        pa.array(["a", "a", "b", "b", "c"]),
        pa.array([1, 2, 3, 4, 5]),
    ],
    names=["keys", "values"],
)

t

pyarrow.Table
keys: string
values: int64
----
keys: [["a","a","b","b","c"]]
values: [[1,2,3,4,5]]

In [13]:
t.group_by("keys").aggregate([("values", "sum")])

pyarrow.Table
values_sum: int64
keys: string
----
values_sum: [[3,7,5]]
keys: [["a","b","c"]]

In [14]:
t = pa.table(
    [
        pa.array(["a", "a", "b", "b", "c"]),
        pa.array([1, 2, 3, 4, 5]),
    ],
    names=["keys", "values"],
)

t.group_by("keys").aggregate([("values", "sum"), ("keys", "count")])

pyarrow.Table
values_sum: int64
keys_count: int64
keys: string
----
values_sum: [[3,7,5]]
keys_count: [[2,2,1]]
keys: [["a","b","c"]]

In [15]:
table_with_nulls = pa.table(
    [pa.array(["a", "a", "a"]), pa.array([1, None, None])], names=["keys", "values"]
)
table_with_nulls.group_by(["keys"]).aggregate(
    [("values", "count", pc.CountOptions(mode="all"))]
)

pyarrow.Table
values_count: int64
keys: string
----
values_count: [[3]]
keys: [["a"]]

In [16]:
table_with_nulls.group_by(["keys"]).aggregate(
    [("values", "count", pc.CountOptions(mode="only_valid"))]
)

pyarrow.Table
values_count: int64
keys: string
----
values_count: [[1]]
keys: [["a"]]

## Working with large data

In [18]:
import pyarrow.dataset as ds

ds.write_dataset(
    birthdays_table,
    "savedir",
    format="parquet",
    partitioning=ds.partitioning(pa.schema([birthdays_table.schema.field("years")])),
)

In [19]:
birthdays_dataset = ds.dataset("savedir", format="parquet", partitioning=["years"])

In [20]:
birthdays_dataset.files

['savedir/1990/part-0.parquet',
 'savedir/1995/part-0.parquet',
 'savedir/2000/part-0.parquet']

In [21]:
import datetime

current_year = datetime.datetime.utcnow().year

In [22]:
for table_chunk in birthdays_dataset.to_batches():
    print("AGES", pc.subtract(current_year, table_chunk["years"]))

AGES [
  32
]
AGES [
  27,
  27
]
AGES [
  22,
  22
]


## Type Metadata

Apache Arrow defines language agnostic column-oriented data structures for array data. These include:

 - Fixed-length primitive types: numbers, booleans, date and times, fixed size binary, decimals, and other values that fit into a given number
 - Variable-length primitive types: binary, string
 - Nested types: list, map, struct, and union
 - Dictionary type: An encoded categorical type


In [24]:
for each in [pa.int32(), pa.string(), pa.binary(), pa.binary(10), pa.timestamp("ms")]:
    print(each, type(each))

int32 <class 'pyarrow.lib.DataType'>
string <class 'pyarrow.lib.DataType'>
binary <class 'pyarrow.lib.DataType'>
fixed_size_binary[10] <class 'pyarrow.lib.FixedSizeBinaryType'>
timestamp[ms] <class 'pyarrow.lib.TimestampType'>


We use the name logical type because the physical storage may be the same for one or more types. For example, int64, float64, and timestamp[ms] all occupy 64 bits per value.

## pandas integration

In [25]:
import pyarrow as pa
import pandas as pd

df = pd.DataFrame({"a": [1, 2, 3]})
df

,a
0,1
1,2
2,3


In [26]:
# Convert from pandas to Arrow
table = pa.Table.from_pandas(df)
table

pyarrow.Table
a: int64
----
a: [[1,2,3]]

In [27]:
# Convert back to pandas
df_new = table.to_pandas()
df_new

,a
0,1
1,2
2,3


In [28]:
# Infer Arrow schema from pandas
schema = pa.Schema.from_pandas(df)
schema

a: int64
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 357

By default pyarrow tries to preserve and restore the .index data as accurately as possible. 

```
pandas -> Arrow Conversion
===========================

------------------------------------------------
Source Type (pandas)  | Destination Type (Arrow)
------------------------------------------------
bool                  | BOOL
(u)int{8,16,32,64}    | (U)INT{8,16,32,64}
float32               | FLOAT
float64               | DOUBLE 
str/unicode           | STRING
pd.Categorical        | DICTIONARY
pd.Timestamp          | TIMESTAMP(unit=ns)
datetime.date         | DATE
datetime.time         | TIME64


```

In [ ]:
: